In [1]:
import numpy as np

import networkx as nx
import matplotlib.pyplot as plt

import os.path as op
import sys
sys.path.append("../")

from src import regmod
from src import utils
from src import solver

import h5py

def get_aggprop(h5dict: h5py._hl.files.File, property: str):
    """
    Get the bundles statistics on whole brain level from the HDF5 file.

    Parameters
    ----------
    h5dict : h5py._hl.files.File 
        The opened HDF5 file.
    property : str
        The property to extract from the HDF5 file.

    Returns
    -------
    ret : np.arrasy
        The array containing the requested property values.
    """

    try:
        ret = np.array(h5dict.get("matrices").get(property))
    except:
        print("Not valid property OR h5 not opened")
    return ret

#### EXAMPLE OF HOW TO GET FIBER LENGTH (for the probability we use `consistency` so need to change that to `length`)

In [7]:
# data_path = "/Users/acionca/data/"
data_path = "../../atlas_data/"

region_file = op.join(data_path, f'F-TRACT-090624/Lausanne2008-33.txt')
scale = 1
connFilename = f'../../atlas_data/fiber_atlas/probconnatlas/wm.connatlas.scale{scale}.h5'
# connFilename = op.join(data_path, f'atlas_data/fiber_atlas/probconnatlas/wm.connatlas.scale{scale}.h5')
hf = h5py.File(connFilename, 'r')

centers = np.array(hf.get('header').get('gmcoords'))

gmregions_names = hf.get('header').get('gmregions')[()]
nb_regions = gmregions_names.shape[0]
length_view = get_aggprop(hf, 'length')


array1 = gmregions_names
array1 = [a.decode('utf-8').lower() for a in array1]

with open(region_file) as f:
    array2 = f.readlines()

array2 = [regionname.replace('.','-').lower().strip() for k, regionname in enumerate(array2)]
for sidx, region in enumerate(array2):
    if 'right' in region:
        array2[sidx] = region.replace('right','rh')
    if 'left' in region:
        array2[sidx] = region.replace('left','lh')

In [11]:
match_indexes = np.zeros(len(array2)) - 1
for k in range(len(array2)):
    for sidx, searchname in enumerate(array1):
        if array2[k] in searchname:
            match_indexes[k] = sidx
        if 'lh-accumbens-area' in array2[k]:
            if 'lh-accumbens' in searchname:
                match_indexes[k] = sidx
        if 'rh-accumbens-area' in array2[k]:
            if 'rh-accumbens' in searchname:
                match_indexes[k] = sidx
                
rightth = np.where(['thal' in tmp for tmp in array1])[0][:7]
leftth = np.where(['thal' in tmp for tmp in array1])[0][7:]

In [ ]:
struct_length = length_view

max_delay = 400
proba_file = op.join(data_path, f'F-TRACT-090624/{max_delay}/probability.txt.gz')

proba_array = np.genfromtxt(proba_file, delimiter=' ')
proba_array = np.array(proba_array).astype(float)

fibers_array = np.zeros_like(proba_array)
for ridx in range(len(fibers_array)):
    for cidx in range(len(fibers_array)):
        
        if (ridx == 83) or (cidx == 83):
            fibers_array[ridx, cidx] = np.nan
        else:
            if ridx == 34 and cidx == 34: fibers_array[ridx, cidx] = struct_length[rightth][:, rightth].mean()
            elif ridx == 34 and cidx == 75: fibers_array[ridx, cidx] = struct_length[rightth][:, rightth].mean()
            elif ridx == 75 and cidx == 34: fibers_array[ridx, cidx] = struct_length[rightth][:, rightth].mean()
            elif ridx == 75 and cidx == 75: fibers_array[ridx, cidx] = struct_length[rightth][:, rightth].mean()
            else:
                fibers_array[ridx, cidx] = struct_length[int(match_indexes[ridx]), int(match_indexes[cidx])]

# NOTE: fibers_array is the re-aligned by regions to match f-tract, matrix of fiber lengths

# Computing connectome (degree) asymmetry

# Investigating the relationship with bundle average lengths

# Comparison between d400 and d50

# Cross-validation by removing bundles

# Computing the connection speeds